# Regression and Other Stories: Bayesian $R^2$

Bayesian R2. See Chapter 11 in Regression and Other Stories.

See also - Andrew Gelman, Ben Goodrich, Jonah Gabry, and Aki Vehtari (2018). R-squared for Bayesian regression models. The American Statistician, 73:307-209 doi:10.1080/00031305.2018.1549100.

----

### Load packages

In [1]:
using StatsPlots, MCMCChains, Turing, DataFrames, GLM

### Function for Bayesian R-squared for turing models

In [ ]:
# TODO
#function bayes_R2(fit)


## Toy data with n=5

In [6]:
x = Array(1:5) .- 3
y = [1.7, 2.6, 2.5, 4.4, 3.8] .- 3
xy = DataFrame(Dict("x"=>x,"y"=>y));

### Lsq fit

In [12]:
fit = lm(@formula(y ~ x), xy)
ols_coef = coef(fit)
yhat = ols_coef[1] .+ ols_coef[2] * x
r = y - yhat
rsq_1 = var(yhat)/var(y)
rsq_2 = var(yhat)/(var(yhat) + var(r))
println(round(rsq_1, digits=3), " ", round(rsq_1, digits=3))

0.766 0.766
